# xarray Sandbox
For playing around during development.

In [26]:
import xarray as xr
import numpy as np
import pysd.utils as utils
import textwrap

# creating dataarrays with data at instantiation

In [103]:
coords={'Dim2': ['D', 'E'], 'Dim1': ['A', 'B', 'C']}
dims = ['Dim1', 'Dim2']
a = xr.DataArray(data=[[.1, .2],[.3,.4],[.5,.6]], coords=coords, dims=dims)
a

<xarray.DataArray (Dim1: 3, Dim2: 2)>
array([[ 0.1,  0.2],
       [ 0.3,  0.4],
       [ 0.5,  0.6]])
Coordinates:
  * Dim2     (Dim2) |S1 'D' 'E'
  * Dim1     (Dim1) |S1 'A' 'B' 'C'

In [107]:
a = xr.DataArray(data=[3, 4, 5], coords={'Dim1': ['A', 'B', 'C']})
print a, '\n'
b = xr.DataArray(data=a, coords={'Dim1': ['A', 'B', 'C']})
print b

<xarray.DataArray (Dim1: 3)>
array([3, 4, 5])
Coordinates:
  * Dim1     (Dim1) |S1 'A' 'B' 'C' 

<xarray.DataArray (Dim1: 3)>
array([3, 4, 5])
Coordinates:
  * Dim1     (Dim1) |S1 'A' 'B' 'C'


In [108]:
a = xr.DataArray(data=[3, 4, 5], coords={'Dim1': ['A', 'B', 'C']})
print a, '\n'
b = xr.DataArray(data=a, coords={'Dim1': ['D', 'E', 'F']})
print b

<xarray.DataArray (Dim1: 3)>
array([3, 4, 5])
Coordinates:
  * Dim1     (Dim1) |S1 'A' 'B' 'C' 

<xarray.DataArray (Dim1: 3)>
array([3, 4, 5])
Coordinates:
  * Dim1     (Dim1) |S1 'D' 'E' 'F'


In [104]:
a = xr.DataArray(data=[3, 4, 5], coords={'Dim1': ['A', 'B', 'C']})
print a
b = xr.DataArray(data=a, coords={'Dim2': ['D', 'E'], 'Dim1': ['A', 'B', 'C']}, dims=['Dim1', 'Dim2'])
print b


<xarray.DataArray (Dim1: 3)>
array([3, 4, 5])
Coordinates:
  * Dim1     (Dim1) |S1 'A' 'B' 'C'


KeyError: 'Dim2'

In [109]:
a = xr.DataArray(data=3, coords={'Dim1': ['A', 'B', 'C']})

ValueError: coordinate Dim1 has dimensions ('Dim1',), but these are not a subset of the DataArray dimensions ()

In [40]:
subs = ['One Dimensional Subscript', 'Second Dimension Subscript', 'Depth 1']
subscript_dict = {'Second Dimension Subscript': ['Column 1', 'Column 2'], 
                  'Third Dimension Subscript': ['Depth 1', 'Depth 2'], 
                  'One Dimensional Subscript': ['Entry 1', 'Entry 2', 'Entry 3']}

coords = utils.make_coord_dict(subs, subscript_dict, terse=False)
dims = [utils.find_subscript_name(subscript_dict, sub) for sub in subs]
shape = [len(coords[dim]) for dim in dims]

text = '1,2; 3,4; 5,6'
text = text.strip(';').replace(' ', '').replace(';',',')

data = np.array([float(s) for s in text.split(',')]).reshape(shape)
datastr = np.array2string(data, separator=',').replace('\n','').replace(' ','')

datastr
a = xr.DataArray(data=data, coords=coords, dims=dims)
a

string = textwrap.dedent("""\
        xr.DataArray(data=%(datastr)s,
                     coords=%(coords)s,
                     dims=%(dims)s )""" % {
        'datastr': datastr,
        'coords': coords,
        'dims': repr(dims)
    })

eval(string)

<xarray.DataArray (One Dimensional Subscript: 3, Second Dimension Subscript: 2, Third Dimension Subscript: 1)>
array([[[ 1.],
        [ 2.]],

       [[ 3.],
        [ 4.]],

       [[ 5.],
        [ 6.]]])
Coordinates:
  * Second Dimension Subscript  (Second Dimension Subscript) |S8 'Column 1' ...
  * One Dimensional Subscript   (One Dimensional Subscript) |S7 'Entry 1' ...
  * Third Dimension Subscript   (Third Dimension Subscript) |S7 'Depth 1'

In [114]:
# modify values directly without index slicing
coords={'Dim2': ['D', 'E'], 'Dim1': ['A', 'B', 'C']}
dims = ['Dim1', 'Dim2']
shape = [len(coords[key]) for key in dims]
val = [3, 2, 1]
a = xr.DataArray(data=np.tile(val, shape), coords=coords, dims=dims)

a

ValueError: conflicting sizes for dimension 'Dim2': length 4 on the data but length 2 on coordinate 'Dim2'

# creating empty dataarrays

In [5]:
# using np.zeros
coords={'Dim2': ['D', 'E'], 'Dim1': ['A', 'B', 'C']}
dims = ['Dim1', 'Dim2']
shape = [len(coords[key]) for key in dims]
a = xr.DataArray(data=np.zeros(shape)*np.NaN, coords=coords, dims=dims)
a.loc[coords]

<xarray.DataArray (Dim1: 3, Dim2: 2)>
array([[ nan,  nan],
       [ nan,  nan],
       [ nan,  nan]])
Coordinates:
  * Dim2     (Dim2) |S1 'D' 'E'
  * Dim1     (Dim1) |S1 'A' 'B' 'C'

In [53]:
# using np.tile
coords={'Dim2': ['D', 'E'], 'Dim1': ['A', 'B', 'C']}
dims = ['Dim1', 'Dim2']
shape = [len(coords[key]) for key in dims]
a = xr.DataArray(data=np.tile(np.NaN, shape), coords=coords, dims=dims)
a.loc[coords]

<xarray.DataArray (Dim1: 3, Dim2: 2)>
array([[ nan,  nan],
       [ nan,  nan],
       [ nan,  nan]])
Coordinates:
  * Dim2     (Dim2) |S1 'D' 'E'
  * Dim1     (Dim1) |S1 'A' 'B' 'C'

# assigning all values to array

In [74]:
# assign all values to a scalar
coords={'Dim2': ['D', 'E'], 'Dim1': ['A', 'B', 'C']}
dims = ['Dim1', 'Dim2']
shape = [len(coords[key]) for key in dims]
a = xr.DataArray(data=np.tile(np.NaN, shape), coords=coords, dims=dims)

a.loc[coords] = 3
a.loc[coords]

<xarray.DataArray (Dim1: 3, Dim2: 2)>
array([[ 3.,  3.],
       [ 3.,  3.],
       [ 3.,  3.]])
Coordinates:
  * Dim2     (Dim2) |S1 'D' 'E'
  * Dim1     (Dim1) |S1 'A' 'B' 'C'

In [75]:
# fill all values
coords={'Dim2': ['D', 'E'], 'Dim1': ['A', 'B', 'C']}
dims = ['Dim1', 'Dim2']
shape = [len(coords[key]) for key in dims]
a = xr.DataArray(data=np.tile(np.NaN, shape), coords=coords, dims=dims)

a.loc[coords] = [1,2],[3,4],[5,6]
a.loc[coords]

<xarray.DataArray (Dim1: 3, Dim2: 2)>
array([[ 1.,  2.],
       [ 3.,  4.],
       [ 5.,  6.]])
Coordinates:
  * Dim2     (Dim2) |S1 'D' 'E'
  * Dim1     (Dim1) |S1 'A' 'B' 'C'

In [76]:
# modify values directly without index slicing
coords={'Dim2': ['D', 'E'], 'Dim1': ['A', 'B', 'C']}
dims = ['Dim1', 'Dim2']
shape = [len(coords[key]) for key in dims]
a = xr.DataArray(data=np.tile(np.NaN, shape), coords=coords, dims=dims)

values = [10,20],[30,40],[50,60]
a.values = np.array(values)
a.loc[coords]

<xarray.DataArray (Dim1: 3, Dim2: 2)>
array([[10, 20],
       [30, 40],
       [50, 60]])
Coordinates:
  * Dim2     (Dim2) |S1 'D' 'E'
  * Dim1     (Dim1) |S1 'A' 'B' 'C'

In [ ]:
# modify values directly without index slicing
coords={'Dim2': ['D', 'E'], 'Dim1': ['A', 'B', 'C']}
dims = ['Dim1', 'Dim2']
shape = [len(coords[key]) for key in dims]
a = xr.DataArray(data=np.tile(np.NaN, shape), coords=coords, dims=dims)

a.loc[]
a.values = np.array(values)
a.loc[coords]

In [9]:
# select a slice
a.loc[{'Dim2':['D']}] 

<xarray.DataArray (Dim1: 3, Dim2: 1)>
array([[10],
       [30],
       [50]])
Coordinates:
  * Dim2     (Dim2) |S1 'D'
  * Dim1     (Dim1) |S1 'A' 'B' 'C'

# Assign to a slice

In [78]:
# assign to a slice (the orientation matters!)
coords={'Dim2': ['D', 'E'], 'Dim1': ['A', 'B', 'C']}
dims = ['Dim1', 'Dim2']
shape = [len(coords[key]) for key in dims]
a = xr.DataArray(data=np.tile(np.NaN, shape), coords=coords, dims=dims)

a.loc[{'Dim2':['D']}] = 1, 2, 3
a.loc[coords]

ValueError: shape mismatch: value array of shape (3,) could not be broadcast to indexing result of shape (1,3)

In [79]:
#assign to a slice with properly aligned array
coords={'Dim2': ['D', 'E'], 'Dim1': ['A', 'B', 'C']}
dims = ['Dim1', 'Dim2']
shape = [len(coords[key]) for key in dims]
a = xr.DataArray(data=np.tile(np.NaN, shape), coords=coords, dims=dims)

a.loc[{'Dim2':['D']}] = [1], [2], [3]
a.loc[coords]


<xarray.DataArray (Dim1: 3, Dim2: 2)>
array([[  1.,  nan],
       [  2.,  nan],
       [  3.,  nan]])
Coordinates:
  * Dim2     (Dim2) |S1 'D' 'E'
  * Dim1     (Dim1) |S1 'A' 'B' 'C'

In [82]:
# assign to opposite dimension
ret = xr.DataArray(data=np.ones([3, 2])*np.NaN,
             coords={'Dim2': ['D', 'E'], 'Dim1': ['A', 'B', 'C']},
             dims=['Dim1', 'Dim2'] )
ret.loc[{'Dim1': ['A']}] = 50, 51
ret.loc[{'Dim1': ['B']}] = 21, 22
ret.loc[{'Dim1': ['C']}] = 3, 4
ret

<xarray.DataArray (Dim1: 3, Dim2: 2)>
array([[ 50.,  51.],
       [ 21.,  22.],
       [  3.,   4.]])
Coordinates:
  * Dim2     (Dim2) |S1 'D' 'E'
  * Dim1     (Dim1) |S1 'A' 'B' 'C'

# update a slice based on an existing dataarray

In [87]:
# add a new slice
coords={'Dim2': ['D', 'E'], 'Dim1': ['A', 'B', 'C']}
dims = ['Dim1', 'Dim2']
shape = [len(coords[key]) for key in dims]
a = xr.DataArray(data=np.tile(np.NaN, shape), coords=coords, dims=dims)

new_coords={'Dim1': ['A', 'B', 'C'], 'Dim2': ['D']}
new_slice = xr.DataArray(data=[[1.1, 2.2, 3.3]], coords=new_coords)

a.update(new_slice)
a

AttributeError: 'DataArray' object has no attribute 'update'

In [84]:
a.loc[new_coords] = new_slice

ValueError: shape mismatch: value array of shape (1,3) could not be broadcast to indexing result of shape (3,1)

In [85]:
a.loc[new_coords].values = new_slice.loc[new_coords].values

ValueError: replacement data must match the Variable's shape

In [93]:
# multiplication works, why not assignment?
coords={'Dim2': ['D', 'E'], 'Dim1': ['A', 'B', 'C']}
dims = ['Dim1', 'Dim2']
a = xr.DataArray(data=[[.1, .2],[.3,.4],[.5,.6]], coords=coords, dims=dims)

new_coords={'Dim1': ['A', 'B', 'C'], 'Dim2': ['D']}
new_slice = xr.DataArray(data=[[1.1, 2.2, 3.3]], coords=new_coords)

a.loc[new_coords] * new_slice.loc[new_coords]
a * new_slice
a + new_slice

<xarray.DataArray (Dim1: 3, Dim2: 1)>
array([[ 1.2],
       [ 2.5],
       [ 3.8]])
Coordinates:
  * Dim2     (Dim2) object 'D'
  * Dim1     (Dim1) |S1 'A' 'B' 'C'

In [28]:
#a2, new_slice2 = xr.align(a,new_slice)
a.loc[new_coords]

<xarray.DataArray (Dim1: 3, Dim2: 1)>
array([[1],
       [1],
       [1]])
Coordinates:
  * Dim2     (Dim2) |S1 'D'
  * Dim1     (Dim1) |S1 'A' 'B' 'C'

In [100]:
coords={'Dim2': ['D', 'E'], 'Dim1': ['A', 'B', 'C']}
dims = ['Dim1', 'Dim2']
shape = [len(coords[key]) for key in dims]
a = xr.DataArray(data=np.tile(np.NaN, shape), coords=coords, dims=dims)
print 'Original Array:\n-----------------\n', a, '\n'

new_coords={'Dim1': ['A', 'B', 'C'], 'Dim2': ['D']}
new_slice = xr.DataArray(data=[[1.1, 2.2, 3.3]], coords=new_coords)
print 'New Slice:\n-----------------\n', new_slice, '\n'

ds = xr.Dataset({'a':a})
ds.update(xr.Dataset({'a':new_slice}))
print 'Updated Array:\n-----------------\n', b['a'], '\n'

Original Array:
-----------------
<xarray.DataArray (Dim1: 3, Dim2: 2)>
array([[ nan,  nan],
       [ nan,  nan],
       [ nan,  nan]])
Coordinates:
  * Dim2     (Dim2) |S1 'D' 'E'
  * Dim1     (Dim1) |S1 'A' 'B' 'C' 

New Slice:
-----------------
<xarray.DataArray (Dim2: 1, Dim1: 3)>
array([[ 1.1,  2.2,  3.3]])
Coordinates:
  * Dim2     (Dim2) |S1 'D'
  * Dim1     (Dim1) |S1 'A' 'B' 'C' 

Updated Array:
-----------------
<xarray.DataArray 'a' (Dim2: 2, Dim1: 3)>
array([[ 1.1,  2.2,  3.3],
       [ nan,  nan,  nan]])
Coordinates:
  * Dim2     (Dim2) object 'D' 'E'
  * Dim1     (Dim1) |S1 'A' 'B' 'C' 



### Using array concatenation

In [2]:
m = xr.DataArray(data=1, 
                 coords={'Dim1': ['A', 'B', 'C'], 'Dim2':['D']})
m

ValueError: coordinate Dim2 has dimensions ('Dim2',), but these are not a subset of the DataArray dimensions ()

In [18]:
m = xr.DataArray(data=[1.1, 1.2, 1.3], 
                 coords={'Dim1': ['A', 'B', 'C'], 'Dim2':['D']})
n = xr.DataArray(data=[2.1, 2.2, 2.3], 
                 coords={'Dim1': ['A', 'B', 'C'], 'Dim2':['E']})

ValueError: coordinate Dim2 has dimensions ('Dim2',), but these are not a subset of the DataArray dimensions ('dim_0',)

In [19]:
m = xr.DataArray(data=[[1.1, 1.2, 1.3]], 
                 coords={'Dim1': ['A', 'B', 'C'], 'Dim2':['D']})
n = xr.DataArray(data=[[2.1, 2.2, 2.3]], 
                 coords={'Dim1': ['A', 'B', 'C'], 'Dim2':['E']})

a = xr.concat([m, n], dim='Dim2')
a

<xarray.DataArray (Dim2: 2, Dim1: 3)>
array([[ 1.1,  1.2,  1.3],
       [ 2.1,  2.2,  2.3]])
Coordinates:
  * Dim1     (Dim1) |S1 'A' 'B' 'C'
  * Dim2     (Dim2) |S1 'D' 'E'

In [24]:
m = xr.DataArray(data=[[[1.1, 1.2, 1.3]]], 
                 coords={'Dim1': ['A', 'B', 'C'], 'Dim2':['D'], 'Dim3':['F']})
n = xr.DataArray(data=[[[2.1, 2.2, 2.3]]], 
                 coords={'Dim1': ['A', 'B', 'C'], 'Dim2':['E'], 'Dim3':['F']})
o = xr.DataArray(data=[[[3.1, 3.2, 3.3]]], 
                 coords={'Dim1': ['A', 'B', 'C'], 'Dim2':['D'], 'Dim3':['G']})
p = xr.DataArray(data=[[[4.1, 4.2, 4.3]]], 
                 coords={'Dim1': ['A', 'B', 'C'], 'Dim2':['E'], 'Dim3':['G']})

a = xr.concat([m,n,o,p], dim=['Dim2', 'Dim3'])

ValueError: the variable 'Dim2' has the same name as one of its dimensions ('concat_dim', 'Dim2'), but it is not 1-dimensional and thus it is not a valid index

In [107]:
m = xr.DataArray(data=[[[1.1, 1.2, 1.3]]], 
                 coords={'Dim1': ['A', 'B', 'C'], 'Dim2':['D'], 'Dim3':['F']})
n = xr.DataArray(data=[[[2.1, 2.2, 2.3]]], 
                 coords={'Dim1': ['A', 'B', 'C'], 'Dim2':['E'], 'Dim3':['F']})
o = xr.DataArray(data=[[[3.1, 3.2, 3.3]]], 
                 coords={'Dim1': ['A', 'B', 'C'], 'Dim2':['D'], 'Dim3':['G']})
p = xr.DataArray(data=[[[4.1, 4.2, 4.3]]], 
                 coords={'Dim1': ['A', 'B', 'C'], 'Dim2':['E'], 'Dim3':['G']})
a = xr.align(m,n,o,p, join='outer')
xr.concat(a, 'Dim1')

<xarray.DataArray (Dim2: 2, Dim3: 2, Dim1: 12)>
array([[[ 1.1,  1.2,  1.3,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
          nan],
        [ nan,  nan,  nan,  nan,  nan,  nan,  3.1,  3.2,  3.3,  nan,  nan,
          nan]],

       [[ nan,  nan,  nan,  2.1,  2.2,  2.3,  nan,  nan,  nan,  nan,  nan,
          nan],
        [ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  4.1,  4.2,
          4.3]]])
Coordinates:
  * Dim2     (Dim2) object 'D' 'E'
  * Dim3     (Dim3) object 'F' 'G'
  * Dim1     (Dim1) |S1 'A' 'B' 'C' 'A' 'B' 'C' 'A' 'B' 'C' 'A' 'B' 'C'

In [60]:
m = xr.DataArray(data=[[[1.1, 1.2, 1.3]]], 
                 coords={'Dim1': ['A', 'B', 'C'], 'Dim2':['D'], 'Dim3':['F']})
n = xr.DataArray(data=[[[2.1, 2.2, 2.3]]], 
                 coords={'Dim1': ['A', 'B', 'C'], 'Dim2':['E'], 'Dim3':['F']})
o = xr.DataArray(data=[[[3.1, 3.2, 3.3]]], 
                 coords={'Dim1': ['A', 'B', 'C'], 'Dim2':['D'], 'Dim3':['G']})
p = xr.DataArray(data=[[[4.1, 4.2, 4.3]]], 
                 coords={'Dim1': ['A', 'B', 'C'], 'Dim2':['E'], 'Dim3':['G']})
a = xr.align(m,n,o,p, join='outer')
xr.concat(a, dim=['Dim2', 'Dim3'])

ValueError: conflicting sizes for dimension 'concat_dim': length 2 on 'concat_dim' and length 4 on <this-array>

In [101]:
m = xr.DataArray(data=[[[1.1, 1.2, 1.3]]], 
                 coords={'Dim1': ['A', 'B', 'C'], 'Dim2':['D'], 'Dim3':['F']})
n = xr.DataArray(data=[[[2.1, 2.2, 2.3]]], 
                 coords={'Dim1': ['A', 'B', 'C'], 'Dim2':['E'], 'Dim3':['F']})
o = xr.DataArray(data=[[[3.1, 3.2, 3.3]]], 
                 coords={'Dim1': ['A', 'B', 'C'], 'Dim2':['D'], 'Dim3':['G']})
p = xr.DataArray(data=[[[4.1, 4.2, 4.3]]], 
                 coords={'Dim1': ['A', 'B', 'C'], 'Dim2':['E'], 'Dim3':['G']})

def xrmerge(das, accept_new=True):
    da = das[0]
    for new_da in das[1:]:
        # Expand both to have same dimensions, padding with NaN
        da, new_da = xr.align(da, new_da, join='outer')
        # Fill NaNs one way or the other re. accept_new
        da = new_da.fillna(da) if accept_new else da.fillna(new_da)
    return da

a = xrmerge([m,n,o,p])
a

<xarray.DataArray (Dim2: 2, Dim3: 2, Dim1: 3)>
array([[[ 1.1,  1.2,  1.3],
        [ 3.1,  3.2,  3.3]],

       [[ 2.1,  2.2,  2.3],
        [ 4.1,  4.2,  4.3]]])
Coordinates:
  * Dim2     (Dim2) object 'D' 'E'
  * Dim3     (Dim3) object 'F' 'G'
  * Dim1     (Dim1) |S1 'A' 'B' 'C'

In [114]:
a.loc[{'Dim1':'A'}]

<xarray.DataArray (Dim2: 2, Dim3: 2)>
array([[ 1.1,  3.1],
       [ 2.1,  4.1]])
Coordinates:
  * Dim2     (Dim2) object 'D' 'E'
  * Dim3     (Dim3) object 'F' 'G'
    Dim1     |S1 'A'

In [115]:
a.loc[{'Dim2':'D'}]

<xarray.DataArray (Dim3: 2, Dim1: 3)>
array([[ 1.1,  1.2,  1.3],
       [ 3.1,  3.2,  3.3]])
Coordinates:
    Dim2     |S1 'D'
  * Dim3     (Dim3) object 'F' 'G'
  * Dim1     (Dim1) |S1 'A' 'B' 'C'

In [116]:
a.loc[{'Dim3':'F'}]

<xarray.DataArray (Dim2: 2, Dim1: 3)>
array([[ 1.1,  1.2,  1.3],
       [ 2.1,  2.2,  2.3]])
Coordinates:
  * Dim2     (Dim2) object 'D' 'E'
    Dim3     |S1 'F'
  * Dim1     (Dim1) |S1 'A' 'B' 'C'

In [117]:
data =[[[ 1.1,  1.2,  1.3],
        [ 3.1,  3.2,  3.3]],

       [[ 2.1,  2.2,  2.3],
        [ 4.1,  4.2,  4.3]]]

b = xr.DataArray(data=data, 
             coords={'Dim1': ['A', 'B', 'C'], 'Dim2':['D', 'E'], 'Dim3':['F', 'G']})

In [126]:
b.values

array([[[ 1.1,  1.2,  1.3],
        [ 3.1,  3.2,  3.3]],

       [[ 2.1,  2.2,  2.3],
        [ 4.1,  4.2,  4.3]]])

In [136]:
for i in b.coords.iteritems():
    print i[0], i[1].values

{key:list(value.values) for key, value in b.coords.iteritems()}

Dim2 ['D' 'E']
Dim3 ['F' 'G']
Dim1 ['A' 'B' 'C']


{'Dim1': ['A', 'B', 'C'], 'Dim2': ['D', 'E'], 'Dim3': ['F', 'G']}

In [131]:
for i in b.indexes.iteritems():
    print i[0], i[1].values

Dim2 ['D' 'E']
Dim3 ['F' 'G']
Dim1 ['A' 'B' 'C']


In [82]:
m = xr.DataArray(data=[[[1.1, 1.2, 1.3]]], 
                 coords={'Dim1': ['A', 'B', 'C'], 'Dim2':['D'], 'Dim3':['F']})
n = xr.DataArray(data=[[[2.1, 2.2, 2.3]]], 
                 coords={'Dim1': ['A', 'B', 'C'], 'Dim2':['E'], 'Dim3':['F']})
o = xr.DataArray(data=[[[3.1, 3.2, 3.3]]], 
                 coords={'Dim1': ['A', 'B', 'C'], 'Dim2':['D'], 'Dim3':['G']})
p = xr.DataArray(data=[[[4.1, 4.2, 4.3]]], 
                 coords={'Dim1': ['A', 'B', 'C'], 'Dim2':['E'], 'Dim3':['G']})
ds = xr.Dataset({'m':m,'n':n,'o':o,'p':p})
ds.to_array()

<xarray.DataArray (variable: 4, Dim2: 2, Dim3: 2, Dim1: 3)>
array([[[[ nan,  nan,  nan],
         [ nan,  nan,  nan]],

        [[ nan,  nan,  nan],
         [ 4.1,  4.2,  4.3]]],


       [[[ 1.1,  1.2,  1.3],
         [ nan,  nan,  nan]],

        [[ nan,  nan,  nan],
         [ nan,  nan,  nan]]],


       [[[ nan,  nan,  nan],
         [ 3.1,  3.2,  3.3]],

        [[ nan,  nan,  nan],
         [ nan,  nan,  nan]]],


       [[[ nan,  nan,  nan],
         [ nan,  nan,  nan]],

        [[ 2.1,  2.2,  2.3],
         [ nan,  nan,  nan]]]])
Coordinates:
  * Dim2      (Dim2) object 'D' 'E'
  * Dim3      (Dim3) object 'F' 'G'
  * Dim1      (Dim1) |S1 'A' 'B' 'C'
  * variable  (variable) |S1 'p' 'm' 'o' 'n'

In [85]:
m = xr.DataArray(data=[[[1.1, 1.2, 1.3]]], 
                 coords={'Dim1': ['A', 'B', 'C'], 'Dim2':['D'], 'Dim3':['F']})
n = xr.DataArray(data=[[[2.1, 2.2, 2.3]]], 
                 coords={'Dim1': ['A', 'B', 'C'], 'Dim2':['E'], 'Dim3':['F']})
o = xr.DataArray(data=[[[3.1, 3.2, 3.3]]], 
                 coords={'Dim1': ['A', 'B', 'C'], 'Dim2':['D'], 'Dim3':['G']})
p = xr.DataArray(data=[[[4.1, 4.2, 4.3]]], 
                 coords={'Dim1': ['A', 'B', 'C'], 'Dim2':['E'], 'Dim3':['G']})
l = [n,o,p]
for j in l:
    xr.concat([m, j])
    break

/Users/houghton/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:11: FutureWarning: the `dim` argument to `concat` will be required in a future version of xarray; for now, setting it to the old default of 'concat_dim'


ValueError: the variable 'Dim2' has the same name as one of its dimensions ('concat_dims', 'Dim2'), but it is not 1-dimensional and thus it is not a valid index

In [ ]:
base = xr.DataArray(data=[[[1.1, 1.2, 1.3]]], 
                    coords={'Dim1': ['A', 'B', 'C'], 'Dim2':['D'], 'Dim3':['F']})
n = xr.DataArray(data=[[[2.1, 2.2, 2.3]]], 
                 coords={'Dim1': ['A', 'B', 'C'], 'Dim2':['E'], 'Dim3':['F']})
o = xr.DataArray(data=[[[3.1, 3.2, 3.3]]], 
                 coords={'Dim1': ['A', 'B', 'C'], 'Dim2':['D'], 'Dim3':['G']})
p = xr.DataArray(data=[[[4.1, 4.2, 4.3]]], 
                 coords={'Dim1': ['A', 'B', 'C'], 'Dim2':['E'], 'Dim3':['G']})

### in pandas 

In [41]:
import pandas as pd
df = pd.DataFrame(index=range(5), columns=['a','b','c','d'])
df2 = pd.DataFrame(index=range(3), columns=['a'], data=range(3))
df.update(df2)
df

,a,b,c,d
0,0,NaN,NaN,NaN
1,1,NaN,NaN,NaN
2,2,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN


In [46]:
df.loc[2:4, 'b'] = 5
df

,a,b,c,d
0,0,NaN,NaN,NaN
1,1,NaN,NaN,NaN
2,2,5,NaN,NaN
3,NaN,5,NaN,NaN
4,NaN,5,NaN,NaN


df.loc[2:4, 'c'] = [5,6,7]
df

### Trying all the types of functions

In [58]:
upstream_funcs = [
    lambda: 5
    lambda: xr.DataArray(data=5, coords={'Dim1': ['A', 'B', 'C']})
    lambda: xr.DataArray(data=5, coords={'Dim1': ['A', 'B', 'C'], 'Dim2':['D']})
    lambda: xr.DataArray(data=5, coords={'Dim1': ['A', 'B', 'C'], 'Dim2':['D', 'E']})
]


,a,b,c,d
0,0,NaN,NaN,NaN
1,1,NaN,NaN,NaN
2,2,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN


In [68]:
a.coords.values()

[<xarray.DataArray 'Dim2' (Dim2: 2)>
 array(['D', 'E'], dtype=object)
 Coordinates:
   * Dim2     (Dim2) object 'D' 'E', <xarray.DataArray 'Dim3' (Dim3: 2)>
 array(['F', 'G'], dtype=object)
 Coordinates:
   * Dim3     (Dim3) object 'F' 'G', <xarray.DataArray 'Dim1' (Dim1: 3)>
 array(['A', 'B', 'C'], 
       dtype='|S1')
 Coordinates:
   * Dim1     (Dim1) |S1 'A' 'B' 'C']

In [73]:
{key:list(val.values) for key, val in a.coords.iteritems()}

{'Dim1': ['A', 'B', 'C'], 'Dim2': ['D', 'E'], 'Dim3': ['F', 'G']}